# 1. Polyhedron modeling

### Part a

### Part b

# 2. Standard form with equality constraints

In [26]:
using JuMP, Clp
p = Model(solver = ClpSolver())

@variable(p, z[1:5])
@expression(p, s1, -z[1] + 6z[2] - z[3] + z[4])
@constraint(p, s1 >= -3)
@constraint(p, 7z[2] + z[4] == 5)
@constraint(p, z[3] + z[4] <= 2)
@constraint(p, -1 <= z[2] <= 2)
@constraint(p, -1 <= z[3] <= 5)
@constraint(p, -2 <= z[4] <= 2)

@objective(p, Max, 3z[1] - z[2])

solve(p)
println(getvalue(z))
println(getobjectivevalue(p))

[8.57143, 0.428571, -1.0, 2.0, 0.0]
25.28571428571429


# 3. Alloy blending

In [24]:
materials = [:ironalloy1, :ironalloy2, :ironalloy3, :copper1, :copper2, :aluminum1, :aluminum2]
chemicals = [:c, :cu, :mn]
cost = Dict( zip( materials, [200, 250, 150, 220, 240, 200, 165]))
availibility = Dict( zip( materials, [400, 300, 600, 500, 200, 300, 250]))

using NamedArrays

compound_mat = [ .025 0.00 .013
                 .030 0.00 .008
                 0.00 .003 0.00
                 0.00 .900 0.00
                 0.00 .960 .040
                 0.00 .004 .012
                 0.00 .006 0.00 ]
compound = NamedArray( compound_mat, (materials,chemicals), ("material", "chemical"))
;

In [38]:
using JuMP, Clp
m = Model(solver = ClpSolver())

@variable(m, tons[materials] >= 0)

#availability constraint for each material
@constraint(m, avail[s in materials], tons[s] <= availibility[s] )

#total tons produced constraint
@constraint(m, sum( tons[s] for s in materials) == 500)

#chemical constraint
@constraint(m, 10 <= sum( tons[s]* compound[s, :c] for s in materials) <= 15 )
@constraint(m, 2 <= sum( tons[s]* compound[s, :cu] for s in materials) <= 3 )
@constraint(m, 6 <= sum( tons[s]* compound[s, :mn] for s in materials) <= 8.25 )

#total cost minimization setup
@expression(m, total_cost, sum( tons[s]*cost[s] for s in materials))
@objective(m, Min, total_cost)


solve(m)
println("The ideal composition for 500 tons of steel is as follows")
println(getvalue(tons))
println("\nThe total cost is: \$", getvalue(total_cost))

The ideal composition for 500 tons of steel is as follows
tons: 1 dimensions:
[ironalloy1] = 400.0
[ironalloy2] = 0.0
[ironalloy3] = 39.77630199231039
[   copper1] = 0.0
[   copper2] = 2.761272282418735
[ aluminum1] = 57.462425725270876
[ aluminum2] = 0.0

The total cost is: $98121.63579168124


# 4. Stigler's diet

In [2]:
# STARTER CODE FOR STIGLER'S DIET PROBLEM
using NamedArrays

# import Stigler's data set
raw = readcsv("stigler.csv")
(m,n) = size(raw)

n_nutrients = 2:n      # columns containing nutrients
n_foods = 3:m          # rows containing food names

nutrients = raw[1,n_nutrients][:]   # the list of nutrients (convert to 1-D array)
foods = raw[n_foods,1][:]           # the list of foods (convert to 1-D array)

# lower[i] is the minimum daily requirement of nutrient i.
lower = Dict( zip(nutrients,raw[2,n_nutrients]) )

# data[f,i] is the amount of nutrient i contained in food f.
data = NamedArray( raw[n_foods,n_nutrients], (foods,nutrients), ("foods","nutrients") );

INFO: Recompiling stale cache file /Users/Tej/.julia/lib/v0.6/Polynomials.ji for module Polynomials.
INFO: Recompiling stale cache file /Users/Tej/.julia/lib/v0.6/NamedArrays.ji for module NamedArrays.


In [11]:
using JuMP, Clp
b = Model(solver = ClpSolver())

@variable(b, amnt[foods] >= 0)

@constraint(b, constr[i in nutrients], sum(amnt[s] * data[s,i] for s in foods) >= lower[i])

@objective(b, Min, sum( amnt[s] for s in foods))

solve(b)

#printing
for i in foods
    if (getvalue(amnt[i]) > 0)
        println(i, " = ", getvalue(amnt[i]))
    end
end
#println(getvalue(amnt))
println("\nThe total cost for the year is: \$", getobjectivevalue(b) * 365)

Wheat Flour (Enriched) = 0.02951906167648827
Liver (Beef) = 0.0018925572907052643
Cabbage = 0.011214435246144865
Spinach = 0.005007660466725203
Navy Beans, Dried = 0.061028563526693246

The total cost for the year is: $39.66173154546625


### Part a

I found the optimal diet to have a total cost lower than that of Stigler (39.67 < 39.93)

The foods that make up the diet are Dried Navy Beans, Spinach, Cabbage, Liver (Beef), and Wheat Flour (Enriched)

In [6]:
# STARTER CODE FOR STIGLER'S DIET PROBLEM
using NamedArrays

# import Stigler's data set
raw = readcsv("stigler_veg.csv")
(m,n) = size(raw)

n_nutrients = 2:n      # columns containing nutrients
n_foods = 3:m          # rows containing food names

nutrients = raw[1,n_nutrients][:]   # the list of nutrients (convert to 1-D array)
foods = raw[n_foods,1][:]           # the list of foods (convert to 1-D array)

# lower[i] is the minimum daily requirement of nutrient i.
lower = Dict( zip(nutrients,raw[2,n_nutrients]) )

# data[f,i] is the amount of nutrient i contained in food f.
data = NamedArray( raw[n_foods,n_nutrients], (foods,nutrients), ("foods","nutrients") );

In [7]:
using JuMP, Clp
b = Model(solver = ClpSolver())

@variable(b, amnt[foods] >= 0)

@constraint(b, constr[i in nutrients], sum(amnt[s] * data[s,i] for s in foods) >= lower[i])

@objective(b, Min, sum( amnt[s] for s in foods))

solve(b)
println(getvalue(amnt))
println("\nThe total cost for the year [ONLY VEGETARIAN FOOD] is: \$", getobjectivevalue(b) * 365)

amnt: 1 dimensions:
[ Wheat Flour (Enriched)] = 0.03545558140888771
[               Macaroni] = 0.0
[Wheat Cereal (Enriched)] = 0.0
[            Corn Flakes] = 0.0
[              Corn Meal] = 0.0
[           Hominy Grits] = 0.0
[                   Rice] = 0.0
[            Rolled Oats] = 0.0
[ White Bread (Enriched)] = 0.0
[      Whole Wheat Bread] = 0.0
[              Rye Bread] = 0.0
[             Pound Cake] = 0.0
[          Soda Crackers] = 0.0
[                   Milk] = 0.0
[  Evaporated Milk (can)] = 0.008591461668763544
[                 Butter] = 0.0
[          Oleomargarine] = 0.0
[                   Eggs] = 0.0
[       Cheese (Cheddar)] = 0.0
[                  Cream] = 0.0
[          Peanut Butter] = 0.0
[             Mayonnaise] = 0.0
[                 Crisco] = 0.0
[                 Apples] = 0.0
[                Bananas] = 0.0
[                 Lemons] = 0.0
[                Oranges] = 0.0
[            Green Beans] = 0.0
[                Cabbage] = 0.011249517312443502
[ 

### Part b

The annual cost for the vegetarian diet is higher than that of the normal diet (39.67 < 39.80).
It contains Dried Navy Beans, Spinach, Cabbage, Evaporated Milk (can), and Wheat Flour (Enriched)

The list of foods is the same with one change, 'Liver (Beef)' is replaced with 'Evaporated Milk (can)'